## Count frequency testing

In [15]:
import string
from collections import Counter

documents = ['Hello, how are you!',
            'Win money, win from home.',
            'Call me now.',
            'Hello, Call hello you tomorrow?']

lower_case_doc = []
for s in documents:
    lower_case_doc.append(s.lower())



no_punc_doc = []
for s in lower_case_doc:
    no_punc_doc.append(s.translate(str.maketrans('','',string.punctuation)))
    
words_doc = []
for s in no_punc_doc:
    words_doc.append(s.split(' '))
    
all_words = []
for s in words_doc:
    all_words.extend(s)
    
print(all_words)

frequency = Counter(all_words)
    
print(frequency)


['hello', 'how', 'are', 'you', 'win', 'money', 'win', 'from', 'home', 'call', 'me', 'now', 'hello', 'call', 'hello', 'you', 'tomorrow']
Counter({'hello': 3, 'you': 2, 'win': 2, 'call': 2, 'how': 1, 'are': 1, 'money': 1, 'from': 1, 'home': 1, 'me': 1, 'now': 1, 'tomorrow': 1})


In [72]:
import pandas as pd

test = {
383:    1,
663:    1,
726:    1,
737:    0,
833:    0
}

test = pd.Series(test)

test[test==0]
prior = {'0': len(test[test == 0])/len(test) , '1': len(test[test == 1])/len(test)}

print(prior)

{'0': 0.4, '1': 0.6}


In [45]:
ham = []
spam = []
    # loop over xtrain and y train
for i,val in y_train.items():
    print(i,val)
    index = i
    if val == 0:
        ham.append(X_train.loc[i])
    else:
        spam.append(X_train.loc[i])
print(ham)

383 1
663 1
726 1
737 0
833 0
[0, 0]


In [46]:
def count_frequency(documents):
    """
    count occurrence of each word in the document set.
    Inputs:
    - documents: list, each entity is a string type SMS message
    Outputs:
    - frequency: a dictionary. The key is the unique words, and the value is the number of occurrences of the word
    
    """

    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    # Step 1: covert all strings into their lower case form
    lower_case_doc = []
    for s in documents:
        lower_case_doc.append(s.lower())
        
    # Step 2: remove all punctuations
    no_punc_doc = []
    for s in lower_case_doc:
        no_punc_doc.append(s.translate(str.maketrans('','',string.punctuation)))
     
    # Step 3: tokenize a sentence, i.e., split a sentence into individual words 
    # using a delimiter. The delimiter specifies what character we will use to identify the beginning 
    # and the end of a word.
    words_doc = []
    for s in no_punc_doc:
        words_doc.append(s.split(' '))
        
    # Step 4: count frequencies. To count the occurrence of each word in the document set. 
    # We can use the `Counter` method from the Python `collections` library for this purpose. 
    # `Counter` counts the occurrence of each item in the list and returns a dictionary with 
    # the key as the item being counted and the corresponding value being the count of that item in the list. 
    all_words = []
    for s in words_doc:
        all_words.extend(s)

    frequency = Counter(all_words)
        
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    return frequency



In [68]:
# test using freq function
import numpy as np

x_train = ['Hello, how are you!',
           'Win money, win from home.',
            'Call me now.',
            'Hello, Call hello you tomorrow?']
y_train = np.array([0,1,1,0])

num_train = len(x_train)


ham = []
spam = []
for i in range(num_train):
    if y_train[i] == 0:
        ham.append(x_train[i])
    else:
        spam.append(x_train[i])
        
print(ham)

freq_ham = count_frequency(ham)

freq_spam = count_frequency(spam)

print(freq_ham)

['Hello, how are you!', 'Hello, Call hello you tomorrow?']
Counter({'hello': 3, 'you': 2, 'how': 1, 'are': 1, 'call': 1, 'tomorrow': 1})


In [76]:
count_all = len(freq_ham)
label_dict = {0: {} , 1: {}} 

# add 0 labels
for key in freq_ham:
    count_x = freq_ham[key]
    label_dict[0][key] = count_x / count_all
    
label_dict[0]['dummy'] = 1/count_all
    
# add 1 labels
for key in freq_spam:
    count_x = freq_spam[key]
    label_dict[1][key] = count_x / count_all   
   
    
print(label_dict)

{0: {'hello': 0.5, 'how': 0.16666666666666666, 'are': 0.16666666666666666, 'you': 0.3333333333333333, 'call': 0.16666666666666666, 'tomorrow': 0.16666666666666666, 'dummy': 0.16666666666666666}, 1: {'win': 0.3333333333333333, 'money': 0.16666666666666666, 'from': 0.16666666666666666, 'home': 0.16666666666666666, 'call': 0.16666666666666666, 'me': 0.16666666666666666, 'now': 0.16666666666666666}}


In [ ]:
from scipy.special import softmax
def predict_label(X_test, prior_prob, cond_prob):
    """
    predict the class labels for the testing set
    Inputs:
    - X_test: an array of shape (num_test,) which stores test data. 
              Each entity is a string type SMS message.
    - prior_prob: a dictionary which stores the prior probability for all categories
              We previously used "prior_prob" as the name of function.  
              Here it is used as a dictionary name.  No confusion should arise.
    - cond_prob: a dictionary whose key is the class label y, and value is another dictionary.
                   In the latter dictionary, the key is word w, and the value is the
                   conditional probability P(X_i = w | y).
    Outputs:
    - predict: an array that stores predicted labels
    - test_prob: an array of shape (num_test, num_classes) which stores the posterior probability of each class
    """

    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    # initialize variables
    num_test = len(X_test)
    num_class = len(prior_prob)
    # empty list to hold test probs calculated each index is a class
    log_prob = np.empty((num_test, num_class))
 
    
    # predict label
    # iterate over all entries in X_test 
    for i in range(num_test):
        # find word count of specific text example
        word_count = count_frequency(X_test[i])
        # compute conditional probability for j classes
        for j in range(num_class):
            # calculate sum of log probabilities
            log_prob[i][j] = math.log(prior_prob['j']) + 
            math.log(compute_test_prob(word_count, prior_prob['j'],cond_prob[j]))
                   
    # predict is argmax of each row of log_prob
    predict = np.argmax(log_prob, axis=1)
            
    # calculate posterior probability, subtract predict for computational ease
    test_prob = softmax(log_prob - predict)
    
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    return predict, test_prob


In [78]:
# unit test case:
# x_test = np.array(['Hello, how are you!'])
# sample outputs:
# y_pred: [0] 
# test_prob: [[0.97958684 0.02041316]]


def compute_test_prob(word_count, prior_cat, cond_cat):
    """
    predict the class label for one test example
    Inputs:
    - word_count: a dictionary which stores the frequencies of each word in a SMS message. 
                  Key is the word, value is the number of its occurrence in that message
    - prior_cat: a scalar. prior probability of a specific category
    - cond_cat: a dictionary. conditional probability of a specific category
    Outputs:
    - prob: discriminant value g_y of a specific category for the test example 
                  (no need of normalization, i.e., not exactly the posterior probability)
    """
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    #initialize conditional probability  
    cond_prob = 1
    # fetch cond prob of each word in i of x_test from cond prob dict
    for key,val in word_count:
        if key in cond_cat:
            cond_prob = cond_prob * (val * cond_cat['key'])
        else:
            cond_prob = cond_prob * (val * cond_cat['dummy'])
     
    prob = prior_cat * cond_prob
    
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    return prob 
    


In [84]:
print(np.full((2, ), 9))



[9 9]
